<a href="https://colab.research.google.com/github/voldrek/Meus_projetos/blob/main/RomaneioVersionDefinitiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re,os,openpyxl,chardet
import numpy as np
from openpyxl import load_workbook

In [ ]:
# Pasta onde os arquivos CSV estão localizados
pasta_arquivos = 'teste'

# Lista para armazenar os DataFrames de cada arquivo CSV
dataframes = []

# Iterar pelos arquivos na pasta
for arquivo in os.listdir(pasta_arquivos):
    if arquivo.endswith('.csv'):
        caminho_arquivo = os.path.join(pasta_arquivos, arquivo)
        df = pd.read_csv(caminho_arquivo, sep='\t', encoding='utf-16')
        dataframes.append(df)

# Concatenar os DataFrames em um único DataFrame
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Salvar o DataFrame concatenado em um novo arquivo CSV
arquivo_concatenado = 'concatenado.csv'
df_concatenado.to_csv(arquivo_concatenado, index=False, sep='\t', encoding='utf-8')

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        result = chardet.detect(file.read())
    return result['encoding']

file_path = arquivo_concatenado
encoding = detect_encoding(file_path)

# Ler os dados do arquivo CSV
df = pd.read_csv(arquivo_concatenado, sep="\t", encoding=encoding)

# Criar um novo arquivo XLSX
arquivo_xlsx = "arquivo.xlsx"
wb = openpyxl.Workbook()
ws = wb.active

# Incluir os cabeçalhos como primeira linha na planilha
for coluna_numero, coluna_nome in enumerate(df.columns, start=1):
    ws.cell(row=1, column=coluna_numero, value=coluna_nome)

# Preencher as células da planilha com os dados do DataFrame
for linha_numero, linha in enumerate(df.values, start=2):
    for coluna_numero, valor in enumerate(linha, start=1):
        ws.cell(row=linha_numero, column=coluna_numero, value=valor)

# Salvar o arquivo XLSX
wb.save(arquivo_xlsx)

df2 = pd.read_excel('arquivo.xlsx')

#Filtrando os Excluir e deletando da tabela
filtro_excluir_novas = df2['Ação'].str.lower().isin(['excluir', 'novas'])

# Filtra o DataFrame para manter apenas as linhas que não atendem ao filtro acima
df2 = df2[~filtro_excluir_novas]


#Aplicando os filtros dos cabeçalhos extras
filtroProduto = df2['Produto'].str.contains('Produto', case=False)
linhasProduto = df2[filtroProduto]
colunaProduto = df2['Produto']
indiceLinhasProduto = linhasProduto.index
df2 = df2.drop(index=indiceLinhasProduto)

#Trasnforma a coluna do Service Id em string
df2['Service Id'] = df2['Service Id'].apply(str)


# Variáveis para padrões de números para extração
padrao_celular = r'(\b\d{11}\b)'
padrao_iccd = r'\b\d{20}\b'
padrao_imei = r'(\b\d{15}\b)'
padrao_nf = r'(\b\d{8}\b)'


resultado = df[df['Service Id'].str.contains(padrao_imei, na=False)]
produtos_encontrados = resultado['Produto']
df2['Aparelhos'] = produtos_encontrados

# Iterar sobre as linhas do DataFrame
for index, row in df2.iterrows():
    service_data = row['Service Id']
    telefone = re.search(padrao_celular, service_data)
    iccd = re.search(padrao_iccd, service_data)
    imei = re.search(padrao_imei, service_data)
    if telefone:
        df2.at[index, 'Numero telefone'] = telefone.group(0)
    if iccd:
      if 'ICCD' in df2.columns:
            df2.at[index, 'ICCD'] = iccd.group(0)
      else:
            df2['ICCD'] = None
            df2.at[index, 'ICCD'] = iccd.group(0)

    if imei:
        if 'IMEI' in df2.columns:
            df2.at[index, 'IMEI'] = imei.group(0)
        else:
            df2['IMEI'] = None
            df2.at[index, 'IMEI'] = imei.group(0)

#Salva os arquivos na nova pasta caso eles sejam localizados
with pd.ExcelWriter('resposta_romaneio.xlsx') as writer:
    df3 = pd.DataFrame().to_excel(writer, sheet_name='Planilha1', index=False)

    if 'ICCD' in df2.columns:
      numero_iccd = df2['ICCD']
      limpeza_iccd = numero_iccd.dropna()
      limpeza_iccd.to_excel(writer, sheet_name='Planilha1', startcol=0, index=False)  # Coluna 'ICCD' na coluna A
    if 'Numero telefone' in df2.columns:
      numero_telefone = df2['Numero telefone']
      limpeza_telefone = numero_telefone.dropna()
      limpeza_telefone.to_excel(writer, sheet_name='Planilha1', startcol=1, index=False)  # Coluna 'Telefone' na coluna b
    if 'Ordem de vendas' in df2.columns:
      numero_ordem = df2['Ordem de vendas']
      limpeza_ordem = numero_ordem.dropna()
      limpeza_ordem.to_excel(writer, sheet_name='Planilha1', startcol=2, index=False)  # Coluna 'Ordem de Venda' na coluna c
    if 'Nota Fiscal' in df2.columns:
      numero_nf = df2['Nota Fiscal']
      limpeza_nf = numero_nf.dropna()
      limpeza_nf.to_excel(writer, sheet_name='Planilha1', startcol=3, index=False) # Coluna 'Nota Fiscal' na coluna c
    if 'IMEI' in df2.columns:
      numero_imei = df2['IMEI']
      limpeza_imei = numero_imei.dropna()
      limpeza_imei.to_excel(writer, sheet_name='Planilha1', startcol=4, index=False)  # Coluna 'IMEI' na coluna E
    if 'Conta de serviço' in df2.columns:
      numero_conta = df2['Conta de serviço'].dropna()
      numero_conta.to_excel(writer, sheet_name='Planilha1', startcol=5, index=False) #Coluna 'Conta de Serviço' na coluna F
    if 'Aparelhos' in df2.columns:
      aparelhos = df2['Aparelhos'].dropna()
      aparelhos.to_excel(writer,sheet_name='Planilha1', startcol=6, index=False) #Coluna 'Aparelhos' na coluna G

#Faz a limpeza dos Excluidos e novos
df3 = pd.read_excel('resposta_romaneio.xlsx')
df3 = df3.dropna(subset=['Numero telefone'])
df3.to_excel('resposta_romaneio.xlsx')

In [162]:
# Filtrar linhas na coluna 'Service Id' que contêm o padrão do IMEI

# Criar uma nova coluna no DataFrame com os valores encontrados



<ipython-input-162-088cf743aeb1>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  resultado = df[df['Service Id'].str.contains(padrao_imei, na=False)]


In [164]:
df2['Aparelhos'].dropna()

11      TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
33      TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
54      TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
77      TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
98      TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
                            ...                     
4485    TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
4506    TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
4529    TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
4553    TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
4576    TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
Name: Aparelhos, Length: 200, dtype: object

In [167]:
df3.head(5)

,ICCD,Numero telefone,Ordem de vendas,Nota Fiscal,IMEI,Conta de serviço,Aparelhos
0,89551062369000249292,1.198513e+10,1.028472e+09,20315621.0,3.527903e+14,374674229,TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
1,89551062369000249284,1.198513e+10,1.028472e+09,20315621.0,3.527903e+14,374674229,TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
2,89551062369000249276,1.198513e+10,1.028472e+09,20315621.0,3.527903e+14,374674229,TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
3,89551062369000249268,1.198513e+10,1.028472e+09,20315621.0,3.527903e+14,374674229,TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
4,89551062369000249250,1.198513e+10,1.028472e+09,20315621.0,3.527903e+14,374674229,TGMO43852000_MOTOROLA XT2335-1 SMARTPHONE GF
